In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [2]:
from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

Loading csv files

In [11]:
#dataset_TIST2015_Checkins: User ID, Venue ID, UTC time, Time zone
#dataset_TIST2015_POIs: Venue ID, Latitude, Longitude, Venue category name, Country code 
#dataset_TIST2015_Cities: Cityname, Latitude, Longitude, Vanue Category, Country code, Country name, City type
#dataset_UbiComp2016_UserProfile_{CITY}: User ID, Gender, Twitter friend count, Twitter follower count

#Scenario 1: Checkins og POI locations

checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkin = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

city_cols = ['city_name', 'citycenter_latitude', 'citycenter_longitude', 'country_code', 'country_name', 'city_type']
cities = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Cities.csv', sep=',', names=city_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')


#Scenario 2: Users og Checkins

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'gender', 'sex']
ratings_cols = ['user_id', 'venue_id', 'latitude', 'longitude', 'category' 'unix_timestamp']

#Scenario3: Users, Checkins og POI Locations
checkin_cols = ['poi_id', 'gender', 'sex']
venue_cols = ['venue_id', 'user_id', 'latitude', 'longitude', 'category' 'unix_timestamp']
user_cols = ['user_id', 'gender']




C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dim1 = cities[['city_name', 'citycenter_longitude', 'citycenter_latitude', 'city_type', 'country_code']].drop(labels=[0], axis=0)

In [16]:
dim2 = checkin[['user_id', 'poi_id', 'timestamp']].drop(labels=[0], axis=0)

In [23]:
dim3 = pois[['poi_id', 'latitude', 'longitude', 'category', 'country_code']].drop(labels=[0], axis=0)

In [25]:
dataset = dim2.merge(dim3, on='poi_id')#.merge(dim1, on='country_code')

In [ ]:
# Merge Users and Checkins. 
# However, our data does not allow us to merge users and checkin, because we only have data for users that have checked in NYC and Tokyo
#movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

In [27]:
#Split into train and test
train_, test = train_test_split(dataset, test_size=0.1)

In [31]:
ROW_COUNT = train_.shape[0]

In [ ]:
EMBEDDING_SIZE = 10
NUM_USERS = dataset['user_id'].nunique()
NUM_POI = dataset['poi_id'].nunique()

In [29]:
dataset.head()

,user_id,poi_id,timestamp,latitude,longitude,category,country_code
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,55.696132,37.557842,Comedy Club,RU
1,190571,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
2,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 05:49:28 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
3,227763,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:10:31 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR
4,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:53:32 +0000 2012,41.029717,28.97442,Other Great Outdoors,TR


In [ ]:
UNIQUE_MOVIE_IDS = dataset['movie_id'].unique()

# Create Subset of Dataset

## Load in the dataset from csv files

In [3]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Function that extracts a subset

In [4]:
# Create a subset of a dataset with size n users
def create_checkin_subset(checkins_table, n):
    subset = np.array([[0,0,0,0],[0,0,0,0]])
    checkins_100_users = checkins_table['user_id'].sample(n=n, random_state=1)
    for i in checkins_100_users:
        subset = np.append(subset, np.asarray(checkins_table[checkins_table['user_id'] == i].values), 0)
    return pd.DataFrame(subset[2::], columns=['user_id', 'poi_id', 'timestamp', 'timezone'])

### Calling the function

In [5]:
# Initialize the subset method
checkins_100_users = create_checkin_subset(checkins, 25)
checkin_data = checkins_100_users.merge(pois, on='poi_id')
df = checkin_data.set_index('user_id').poi_id.str.get_dummies(',')
df = df.groupby('user_id').max()

## Extracting all of the users and the pois

In [6]:
listofusers = pd.DataFrame(checkin_data, columns= ['user_id']).groupby('user_id').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['poi_id', 'latitude', 'longitude']).groupby('poi_id').max().sample(frac=1)

In [7]:
print(listofpois)

                           latitude   longitude
poi_id                                         
517788c4498e66dc9a284e7b   37.75565   29.114521
4eb57f05775b544c2752966b -25.451978  -49.243192
4bedb3832c082d7f095d2f42 -25.425964  -49.267637
4c1f7d15eac020a13d244bc2  50.450482   30.594821
4bc50d63ccbcef3ba40fe6d2  35.685588  139.699981
...                             ...         ...
4c73882bf3279c74a2dfb32d  35.439519  139.644728
4b570e89f964a520b22428e3  34.985495  135.758593
4d5215769b27721e6f9cbe46  -6.151155  106.892059
49d77968f964a520275d1fe3  41.888485  -87.635236
4c41836fd691c9b65ade8c0a  35.851371  139.786674

[2896 rows x 2 columns]


In [8]:
userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()

In [9]:
userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])

In [ ]:
#REDUNDANT, i think

rows_list = []
for i in range(len(poidataframe)):
    longitude = listofpois[listofpois.loc[i, "Poi"]]
    rows_list.append(dict1)

latlong = pd.DataFrame(rows_list) 
    

In [11]:
dot = userdataframe.merge(poidataframe, how='cross')

In [12]:
print(dot)

       Users                       Poi
0      10427  517788c4498e66dc9a284e7b
1      10427  4eb57f05775b544c2752966b
2      10427  4bedb3832c082d7f095d2f42
3      10427  4c1f7d15eac020a13d244bc2
4      10427  4bc50d63ccbcef3ba40fe6d2
...      ...                       ...
72395  25408  4c73882bf3279c74a2dfb32d
72396  25408  4b570e89f964a520b22428e3
72397  25408  4d5215769b27721e6f9cbe46
72398  25408  49d77968f964a520275d1fe3
72399  25408  4c41836fd691c9b65ade8c0a

[72400 rows x 2 columns]


## Extracting ground_truth from incidence matrix

In [13]:
rows_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':temp}
    rows_list.append(dict1)

groundtruth = pd.DataFrame(rows_list) 


In [ ]:
result = pd.concat([dot, groundtruth], axis=1)

Define the model

In [35]:
class EmbModel(Model):
    def __init__(self):
        super(EmbModel, self).__init__()
        self.model = self.init_model()
        self.d_steps = 16
        
    def call(self, inputs):
        return
    
    def init_model(self):
        poi_latitude_input = keras.Input(shape=(1,), name='poi_latitude')
        poi_longitude_input = keras.Input(shape=(1,), name='poi_longitude')
        poi_concat_input = tf.keras.layers.Concatenate(axis=-1)([poi_latitude_input, poi_longitude_input])
        poi_emb = layers.Embedding(output_dim=10, input_dim=1, input_length=1, name='poi_emb')(poi_concat_input)
    
        user_input = keras.Input(shape=(1,), name='user_id')
        user_emb = layers.Embedding(output_dim=10, input_dim=1, input_length=1, name='user_emb')(user_input)
        #print(user_emb.shape)
    
        dot = layers.Dot(axes=(2))([poi_emb, user_emb])
        model = Model([poi_latitude_input, poi_longitude_input], poi_emb)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        print("Data breakdown: \n")
        
        if len(data) == 3:
            print("Case: 3")
            real_data, labels, sample_weight = data
            print("Realdata: ")
            print(real_data)
            print("\nLabels:")
            print(labels)
            print("\n")
        else:
            print("Case: 2")
            sample_weight = None
            real_data, labels = data
    
            print("Realdata: ")
            print(real_data)
            print("\nLabels:")
            print(labels)
            print('\n')
        
        batch_size = real_data.get_shape()[0]
        print("\nBatch Size:")
        print(batch_size)
        print("\n")
        
        for i in range(self.d_steps):
            
            with tf.GradientTape() as tape:
                with tf.compat.v1.Session() as sess:  
                    print(real_data[0])
                    print(real_data[0][1])
                    print(real_data[0][2])
                dotproduct = self.model([real_data[1], real_data[2]])
                print(dotproduct)
                
                # Loss function = ||S-GroundTruth|| 
                loss = abs(dotproduct - labels)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))

In [36]:
model = EmbModel()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_latitude (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_longitude (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 2)            0           poi_latitude[0][0]               
                                                                 poi_longitude[0][0]              
__________________________________________________________________________________________________
poi_emb (Embedding)             (None, 2, 10)        10          concatenate_5[0][0]   

In [16]:
print(listofpois)

                           latitude   longitude
poi_id                                         
517788c4498e66dc9a284e7b   37.75565   29.114521
4eb57f05775b544c2752966b -25.451978  -49.243192
4bedb3832c082d7f095d2f42 -25.425964  -49.267637
4c1f7d15eac020a13d244bc2  50.450482   30.594821
4bc50d63ccbcef3ba40fe6d2  35.685588  139.699981
...                             ...         ...
4c73882bf3279c74a2dfb32d  35.439519  139.644728
4b570e89f964a520b22428e3  34.985495  135.758593
4d5215769b27721e6f9cbe46  -6.151155  106.892059
49d77968f964a520275d1fe3  41.888485  -87.635236
4c41836fd691c9b65ade8c0a  35.851371  139.786674

[2896 rows x 2 columns]


In [17]:
print([poidataframe.loc[0, "Poi"]])
print(listofpois.loc['4c576c0ccc96c9b6a70f792e']['latitude'])

['517788c4498e66dc9a284e7b']
37.843275


In [18]:
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['latitude']
    longitude = listofpois.loc[temp]['longitude']
    dict1 = {'latitude':latitude, 'longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list) 

## Creating dataset

In [19]:
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['latitude'] = pd.to_numeric(latlong['latitude'])
latlong['longitude'] = pd.to_numeric(latlong['longitude'])
dataset = pd.concat([userdot, latlong], axis=1)
print(dataset.dtypes)

Users          int64
latitude     float64
longitude    float64
dtype: object


In [20]:
dataset_numpy = dataset.to_numpy()
labels_numpy = groundtruth.to_numpy()

In [21]:
#print(dataset_numpy)
print(dataset_numpy[0][0])
print(type(dataset_numpy[0][0]))
print(type(dataset_numpy[0][1]))
print(type(dataset_numpy[0][2]))
#print(labels_numpy)

10427.0
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [22]:
dataset = tf.convert_to_tensor(
    dataset_numpy, dtype=None, dtype_hint=None, name=None)
labels = tf.convert_to_tensor(
    labels_numpy, dtype=None, dtype_hint=None, name=None)

In [34]:
import sys
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)

model.compile(
    optimizer
)

#Train_data, [dataset.user_id, dataset.poi_id]. Label: ground_truth
model.fit(dataset, labels, epochs = 50)

Epoch 1/50
Data breakdown: 

Case: 2
Realdata: 
Tensor("IteratorGetNext:0", shape=(None, 3), dtype=float64)

Labels:
Tensor("IteratorGetNext:1", shape=(None, 1), dtype=int64)



Batch Size:
None


Tensor("strided_slice:0", shape=(3,), dtype=float64)
Tensor("strided_slice_2:0", shape=(), dtype=float64)
Tensor("strided_slice_4:0", shape=(), dtype=float64)
Tensor("functional_7/poi_emb/embedding_lookup/Identity_1:0", shape=(3, 2, 10), dtype=float32)


TypeError: in user code:

    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\lasse\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
  

Train the model

In [ ]:
callbacks = [keras.callbacks.EarlyStopping('val_loss', patience=10),
             keras.callbacks.ModelCheckpoint('besttest.h5', save_best_only=True)]

history = model.fit([train.movie_id, train.user_id],train.rating, batch_size=100,
                              epochs =50, validation_data = ([test.movie_id, test.user_id],test.rating),
                              verbose = 1, 
                              callbacks = callbacks)

In [ ]:
user_model.predict([np.array([10]), np.array([3])])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import matplotlib.pyplot as plt
plt.plot(history.history['loss'] , 'g')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train'], loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
print('test')

In [9]:
print('test')

test
